In [4]:
import pickle
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

parameter

In [2]:
C = 1.0
n_splits = 5
output_file = f'model_C={C}.bin'

data preparation

In [5]:
import kagglehub

path = kagglehub.dataset_download("alexteboul/diabetes-health-indicators-dataset")
print("Path to dataset files:", path)
csv_path = os.path.join(path, "diabetes_binary_health_indicators_BRFSS2015.csv")
df = pd.read_csv(csv_path)
df.head()

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/alexteboul/diabetes-health-indicators-dataset/versions/1


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [6]:
df.columns = df.columns.str.lower().str.replace(' ','_')
df = df.astype(int)
df['sex']=df['sex'].map({0:'female',1:'male'})

In [7]:
df_full_train,df_test = train_test_split(df,test_size=0.2,random_state=1)

In [8]:
categorical=['highbp', 'highchol', 'cholcheck', 'smoker',
       'stroke', 'heartdiseaseorattack', 'physactivity', 'fruits', 'veggies',
       'hvyalcoholconsump', 'anyhealthcare', 'nodocbccost', 'genhlth', 'diffwalk', 'sex', 'age', 'education',
       'income']
numerical = ['bmi','menthlth', 'physhlth']

training

In [9]:
def train(df_train,y_train,C=1.0):
    dict = df_train[categorical + numerical].to_dict(orient = 'records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dict)

    rf = RandomForestClassifier(n_estimators=200,
                            max_depth= 10,
                            min_samples_leaf= 5,
                            random_state=1)
    rf.fit(X_train, y_train)

    return rf,model

In [10]:
def predict(df,rf,model):
    dict = df[categorical + numerical].to_dict(orient='records')
    X = dv.transform(dict)
    y_pred = model.predict_proba(X)[:,1]
    return y_pred

In [12]:
df_full_train.head()

,diabetes_binary,highbp,highchol,cholcheck,bmi,smoker,stroke,heartdiseaseorattack,physactivity,fruits,...,anyhealthcare,nodocbccost,genhlth,menthlth,physhlth,diffwalk,sex,age,education,income
164505,0,1,0,1,38,1,0,0,1,1,...,1,0,3,0,0,0,female,12,5,8
75594,0,0,1,1,25,1,0,0,1,0,...,1,0,3,15,3,1,female,9,6,7
86175,0,1,0,1,32,0,0,0,1,0,...,1,0,3,0,0,0,male,9,4,6
194093,0,0,0,1,19,1,0,0,1,1,...,1,0,3,0,0,1,male,13,6,6
187773,1,1,1,1,31,1,0,0,1,1,...,1,0,2,0,2,0,female,10,6,8


In [14]:
df_full_train.iloc[0]

diabetes_binary              0
highbp                       1
highchol                     0
cholcheck                    1
bmi                         38
smoker                       1
stroke                       0
heartdiseaseorattack         0
physactivity                 1
fruits                       1
veggies                      1
hvyalcoholconsump            0
anyhealthcare                1
nodocbccost                  0
genhlth                      3
menthlth                     0
physhlth                     0
diffwalk                     0
sex                     female
age                         12
education                    5
income                       8
Name: 164505, dtype: object

In [ ]:
people = {
    'diabetes_binary':0,
    'highbp':1,
    'highchol':0,
    'cholcheck':1,
    'bmi':38,
    'smoker':1,
    'stroke':0,
    'heartdiseaseorattack':0,
    'physactivity':1,
    'fruits':1,
    'veggies':1,
    'hvyalcoholconsump':0,
    'anyhealthcare':1,
    'nodocbccost':0,
    'genhlth':3,
    'menthlth':0,
    'physhlth':0,
    'diffwalk':0,
    'sex':'female',
    'age':12,
    'education':5,
    'income':8
}